# Activity 3

In [25]:
import numpy as np
from scipy.optimize import least_squares
from scipy.stats import norm
import matplotlib.pyplot as plt

# Data generation function
def generate_data(samples=1000, p=0.5, noise_std=0.1):
    X = np.random.binomial(1, p, samples)
    # Add some noise and non-linearity to make the problem more interesting
    y = X + np.sin(X * 2) + np.random.normal(0, noise_std, samples)
    return X.reshape(-1, 1), y

# Polynomial basis functions
def polynomial_basis(X, degree=11):
    X = X.flatten()
    return np.vstack([X**i for i in range(degree+1)]).T

# Gaussian basis functions
def gaussian_basis(X, num_centers=11):
    X = X.flatten()
    centers = np.linspace(np.min(X), np.max(X), num_centers)
    width = (centers[1] - centers[0]) if len(centers) > 1 else 1.0
    return np.vstack([norm.pdf(X, loc=center, scale=width) for center in centers]).T

# Sigmoidal basis functions
def sigmoid_basis(X, num_centers=11):
    X = X.flatten()
    centers = np.linspace(np.min(X), np.max(X), num_centers)
    width = (centers[1] - centers[0]) if len(centers) > 1 else 1.0
    return np.vstack([1 / (1 + np.exp(-(X - center) / width)) for center in centers]).T

# Least squares fitting function
def fit_model(X, y, basis_func, **basis_params):
    phi = basis_func(X, **basis_params)
    # Use np.linalg.lstsq for direct solution of least squares
    weights, residuals, rank, s = np.linalg.lstsq(phi, y, rcond=None)
    return weights

# Prediction function
def predict(X, weights, basis_func, **basis_params):
    phi = basis_func(X, **basis_params)
    return phi @ weights

# Bayesian Linear Regression
def bayesian_linear_regression(X, y, basis_func, alpha=1.0, beta=1.0, **basis_params):
    phi = basis_func(X, **basis_params)
    S_N_inv = alpha * np.eye(phi.shape[1]) + beta * phi.T @ phi
    S_N = np.linalg.inv(S_N_inv)
    m_N = beta * S_N @ phi.T @ y
    return m_N, S_N

# Function to evaluate model performance
def evaluate_model(X_train, y_train, X_test, y_test, weights, basis_func, **basis_params):
    y_pred_train = predict(X_train, weights, basis_func, **basis_params)
    y_pred_test = predict(X_test, weights, basis_func, **basis_params)
    
    train_mse = np.mean((y_train - y_pred_train) ** 2)
    test_mse = np.mean((y_test - y_pred_test) ** 2)
    
    return train_mse, test_mse

# Main execution
if __name__ == "__main__":
    # Generate data
    np.random.seed(42)
    X, y = generate_data(samples=1000)
    
    # Split into train and test sets
    train_idx = np.random.choice(len(X), int(0.8*len(X)), replace=False)
    test_idx = np.array(list(set(range(len(X))) - set(train_idx)))
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    # Dictionary to store results
    results = {}
    
    # 1. Polynomial Model
    poly_weights = fit_model(X_train, y_train, polynomial_basis)
    poly_train_mse, poly_test_mse = evaluate_model(
        X_train, y_train, X_test, y_test, 
        poly_weights, polynomial_basis
    )
    results['polynomial'] = (poly_train_mse, poly_test_mse)
    
    # 2. Gaussian Model
    gauss_weights = fit_model(X_train, y_train, gaussian_basis)
    gauss_train_mse, gauss_test_mse = evaluate_model(
        X_train, y_train, X_test, y_test, 
        gauss_weights, gaussian_basis
    )
    results['gaussian'] = (gauss_train_mse, gauss_test_mse)
    
    # 3. Sigmoidal Model
    sigmoid_weights = fit_model(X_train, y_train, sigmoid_basis)
    sigmoid_train_mse, sigmoid_test_mse = evaluate_model(
        X_train, y_train, X_test, y_test, 
        sigmoid_weights, sigmoid_basis
    )
    results['sigmoid'] = (sigmoid_train_mse, sigmoid_test_mse)
    
    # Print results
    print("\nModel Performance Summary:")
    print("-" * 50)
    for model_name, (train_mse, test_mse) in results.items():
        print(f"{model_name.capitalize()} Model:")
        print(f"Training MSE: {train_mse:.4f}")
        print(f"Test MSE: {test_mse:.4f}\n")
        
    # Bayesian Linear Regression for all three models
    print("\nBayesian Linear Regression Results:")
    print("-" * 50)
    for basis_func in [polynomial_basis, gaussian_basis, sigmoid_basis]:
        m_N, S_N = bayesian_linear_regression(X_train, y_train, basis_func)
        y_pred = predict(X_test, m_N, basis_func)
        mse = np.mean((y_test - y_pred) ** 2)
        print(f"{basis_func.__name__}:")
        print(f"Test MSE: {mse:.4f}\n")


Model Performance Summary:
--------------------------------------------------
Polynomial Model:
Training MSE: 0.0096
Test MSE: 0.0103

Gaussian Model:
Training MSE: 0.0096
Test MSE: 0.0103

Sigmoid Model:
Training MSE: 0.0096
Test MSE: 0.0103


Bayesian Linear Regression Results:
--------------------------------------------------
polynomial_basis:
Test MSE: 0.0103

gaussian_basis:
Test MSE: 0.0103

sigmoid_basis:
Test MSE: 0.0104

